# Data Imputation

In [ ]:
# # iterative imputation transform
# from numpy import isnan
# from pandas import read_csv
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer

# data = df_new.values
# ix = [i for i in range(data.shape[1]) if i != 0]
# X, y = data[:, ix], data[:, 0]
# # print total missing
# print('Missing: %d' % sum(isnan(X).flatten()))
# # define imputer
# imputer = IterativeImputer(max_iter = 50, skip_complete = True)
# # fit on the dataset
# imputer.fit(X)
# # transform the dataset
# Xtrans = imputer.transform(X)
# # print total missing
# print('Missing: %d' % sum(isnan(Xtrans).flatten()))
     
# Missing: 521
# Missing: 0

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.pipeline import Pipeline
# # define modeling pipeline
# model = RandomForestClassifier()
# imputer = IterativeImputer()
# pipeline = Pipeline(steps=[('i', imputer), ('m', model)])

# Data Visualizaition

In [ ]:
df.columns = df.columns.str.lower()

numeric_cols = [
    'extremely_nervous', 'irrational_thoughts_risk_lasting',
    'coping_strategy', 'kinesiophobia_physical_exercise',
    'kinesiophobia_pain_stop', 'neck_pain_intensity', 'low_back_pain_intensity',
    'arm_left_pain_intensity', 'arm_right_pain_intensity', 'leg_left_pain_intensity',
    'leg_right_pain_intensity', 'decreased_mobility','weightloss_per_year',
]
df[numeric_cols] = df[numeric_cols].astype(int)

print(df.dtypes)

## removed columns:   ['Workoverload', 'Relationship_with_colleagues', 'Irrational_thoughts_work', 'Trauma', 'Incoordination', 'working_ability']

binary_cols = [
    'fever', 'sick_leave', 'earlier_hospitalization', 'familiy_history',
    'depression', 'stress', 'uses_analgesics', 'uses_corticosteroids', 'serious_disease',
    'neurogenic_signals', 'continuous_pain', 'nocturnal_pain', 'loss_muscle_strength',
    'failure_symptoms', 'paidwork'
]
    
df[binary_cols] = df[binary_cols].astype(int)##à#.astype('bool')

In [ ]:
import numpy as np


corr = df.drop(columns=['patient_id', 'age']).corr()
# Create a mask for the upper triangle
mask = np.triu(np.ones(corr.shape), k=1).astype(bool)

# Apply the mask
corr_masked = corr.mask(mask)

# Style the output
corr_masked.style.background_gradient(cmap='coolwarm')

In [ ]:
strong_corr = corr_masked.unstack().dropna()

# Filter for absolute correlations > 0.5 (excluding 1.0)
strong_corr = strong_corr[(strong_corr.abs() > 0.5) & (strong_corr.abs() < 1.0)]

# Sort by strength
strong_corr = strong_corr.sort_values(key=np.abs, ascending=False)
print(strong_corr)

In [ ]:
df[numeric_cols].hist(figsize=(10, 9), bins=20, edgecolor="black")
plt.suptitle("Histograms of Continous features", fontsize=16)
plt.show()

plt.figure(figsize=(10, 9))

for i, col in enumerate(numeric_cols, 1):
    plt.subplot(4, 4, i)  # Adjust grid size based on the number of numeric columns
    sns.boxplot(y=df[col])
    plt.title(col)

plt.suptitle("Boxplots of Continuous Features", fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
df[numeric_cols].describe()